#### [교차검증 (Cross Validation)]
- 적은 데이터셋으로 안정적이고 신뢰성 있는 모델 평가를 위한 방법
- 학습 데이터셋을 K개 분할 후 매번 다른 데이터로 검증 진행
- 교차검증 후 모델의 일반화 성능으로 여김

[1] 모듈 로딩 및 데이터 준비 <hr>

In [211]:
# ----------------------------------------------------
# [1-1] 모듈 로딩
# ----------------------------------------------------
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 머신러닝 
from sklearn.model_selection import KFold, StratifiedKFold  # 교차검증
from sklearn.neighbors import KNeighborsClassifier          # 학습 알고리즘(KNN)

# ----------------------------------------------------
# [1-2] 데이터 준비
# ----------------------------------------------------
FILE_NAME = '../Data/iris.csv'
irisDF = pd.read_csv(FILE_NAME)
display(irisDF.head(3))

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 전처리 및 학습 준비

In [212]:
# ----------------------------------------------------
# [2-1] 품종컬럼 자료형 변환
# ----------------------------------------------------
pd.options.mode.copy_on_write = True
irisDF.variety = irisDF.variety.astype('category')
print(irisDF.info())

# ----------------------------------------------------
# [2-2] 피쳐와 타겟 분리
# ----------------------------------------------------
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

print(f"featureDF : {featureDF.shape}, targetSR : {targetSR.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   sepal.length  150 non-null    float64 
 1   sepal.width   150 non-null    float64 
 2   petal.length  150 non-null    float64 
 3   petal.width   150 non-null    float64 
 4   variety       150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB
None
featureDF : (150, 4), targetSR : (150,)


[3] 교차검증 <HR>

In [213]:
# ----------------------------------------------------
# [3-1] K-Fold 교차검증
# ----------------------------------------------------
# K-Fold 인스턴스 생성
k_fold = KFold(
    random_state=7,
    shuffle=True
)
print(f"k_fold : {k_fold}\n")

resultDF = pd.DataFrame(columns=['neighbors', 'train', 'valid', 'diff'])
MAX_N = 31
for n in range(1, MAX_N):
    # K개 교차검증 진행
    # -> 학습용 k-1/k 인덱스, 검증용 1/k 인덱스
    t_scores, v_scores = [], []

    for train_index, valid_index in k_fold.split(featureDF):
        
        # 학습 진행
        train_feature, train_target = featureDF.iloc[train_index], targetSR[train_index]
        valid_feature, valid_target = featureDF.iloc[valid_index], targetSR[valid_index]
        # print(f"train_target별 비율 : {round(train_target.value_counts()/train_target.shape[0], 2)}\n")
        # print(f"valid_target별 비율 : {round(valid_target.value_counts()/valid_target.shape[0], 2)}")


        k_model = KNeighborsClassifier(n_neighbors=n)
        k_model.fit(train_feature, train_target)

        # 검증 진행

        train_score = k_model.score(train_feature, train_target)
        t_scores.append(train_score)

        valid_score = k_model.score(valid_feature, valid_target)
        v_scores.append(valid_score)

    # K-Fold 진행 후 성능 평균
    t_scores_mean = sum(t_scores)/len(t_scores)
    v_scores_mean = sum(v_scores)/len(v_scores)
    print(f"n_neighbors가 {n:2d}일 때 t-scores(train 성능) 평균 : {t_scores_mean:.3f}")
    print(f"n_neighbors가 {n:2d}일 때 v-scores(valid 성능) 평균 : {v_scores_mean:.3f}\n")
    resultDF.loc[resultDF.shape[0]] = [n, t_scores_mean, v_scores_mean, abs(t_scores_mean - v_scores_mean)]

k_fold : KFold(n_splits=5, random_state=7, shuffle=True)

n_neighbors가  1일 때 t-scores(train 성능) 평균 : 1.000
n_neighbors가  1일 때 v-scores(valid 성능) 평균 : 0.960

n_neighbors가  2일 때 t-scores(train 성능) 평균 : 0.975
n_neighbors가  2일 때 v-scores(valid 성능) 평균 : 0.967

n_neighbors가  3일 때 t-scores(train 성능) 평균 : 0.963
n_neighbors가  3일 때 v-scores(valid 성능) 평균 : 0.960

n_neighbors가  4일 때 t-scores(train 성능) 평균 : 0.963
n_neighbors가  4일 때 v-scores(valid 성능) 평균 : 0.967

n_neighbors가  5일 때 t-scores(train 성능) 평균 : 0.970
n_neighbors가  5일 때 v-scores(valid 성능) 평균 : 0.960

n_neighbors가  6일 때 t-scores(train 성능) 평균 : 0.973
n_neighbors가  6일 때 v-scores(valid 성능) 평균 : 0.953

n_neighbors가  7일 때 t-scores(train 성능) 평균 : 0.972
n_neighbors가  7일 때 v-scores(valid 성능) 평균 : 0.967

n_neighbors가  8일 때 t-scores(train 성능) 평균 : 0.978
n_neighbors가  8일 때 v-scores(valid 성능) 평균 : 0.960

n_neighbors가  9일 때 t-scores(train 성능) 평균 : 0.977
n_neighbors가  9일 때 v-scores(valid 성능) 평균 : 0.960

n_neighbors가 10일 때 t-scores(train 성능) 평균 : 0.975
n_

In [214]:
# 훈련용과 검증용 데이터의 성능 평균 차이가 낮은 거부터 오름차순 정렬
display(resultDF.sort_values(by=['diff', 'neighbors']))

,neighbors,train,valid,diff
27,28.0,0.953333,0.953333,0.000000
2,3.0,0.963333,0.960000,0.003333
28,29.0,0.956667,0.960000,0.003333
10,11.0,0.976667,0.973333,0.003333
12,13.0,0.976667,0.973333,0.003333
3,4.0,0.963333,0.966667,0.003333
6,7.0,0.971667,0.966667,0.005000
13,14.0,0.978333,0.973333,0.005000
16,17.0,0.973333,0.966667,0.006667
1,2.0,0.975000,0.966667,0.008333


In [ ]:
# ----------------------------------------------------
# [3-2] Stratifield K-Fold 교차검증
#       => .split(2D_피쳐, 1D_타겟) : 타겟의 클래스/라벨 비율 계산 및 적용
#       => 분류 모델일 때  
# ----------------------------------------------------
# K-Fold 인스턴스 생성
k_fold = StratifiedKFold(
    random_state=7,
    shuffle=True
)
print(f"k_fold : {k_fold}\n")

resultDF = pd.DataFrame(columns=['neighbors', 'train', 'valid', 'diff'])
MAX_N = 31
for n in range(1, MAX_N):
    # K개 교차검증 진행
    # -> 학습용 k-1/k 인덱스, 검증용 1/k 인덱스
    t_scores, v_scores = [], []

    for train_index, valid_index in k_fold.split(featureDF, targetSR):
        
        # 학습 진행
        train_feature, train_target = featureDF.iloc[train_index], targetSR[train_index]
        valid_feature, valid_target = featureDF.iloc[valid_index], targetSR[valid_index]
        print(f"train_target별 비율 : {round(train_target.value_counts()/train_target.shape[0], 2)}\n")
        print(f"valid_target별 비율 : {round(valid_target.value_counts()/valid_target.shape[0], 2)}")


        k_model = KNeighborsClassifier(n_neighbors=n)
        k_model.fit(train_feature, train_target)

        # 검증 진행
        train_score = k_model.score(train_feature, train_target)
        t_scores.append(train_score)

        valid_score = k_model.score(valid_feature, valid_target)
        v_scores.append(valid_score)

    # K-Fold 진행 후 성능 평균
    t_scores_mean = sum(t_scores)/len(t_scores)
    v_scores_mean = sum(v_scores)/len(v_scores)
    print(f"n_neighbors가 {n:2d}일 때 t-scores(train 성능) 평균 : {t_scores_mean:.3f}")
    print(f"n_neighbors가 {n:2d}일 때 v-scores(valid 성능) 평균 : {v_scores_mean:.3f}\n")
    resultDF.loc[resultDF.shape[0]] = [n, t_scores_mean, v_scores_mean, abs(t_scores_mean - v_scores_mean)]

k_fold : StratifiedKFold(n_splits=5, random_state=7, shuffle=True)

train_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64

valid_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64
train_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64

valid_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64
train_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64

valid_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64
train_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64

valid_target별 비율 : variety
Setosa        0.33
Versicolor    0.33
Virginica     0.33
Name: count, dtype: float64
train_target별 비율 : variety
Setos

In [216]:
# 훈련용과 검증용 데이터의 성능 평균 차이가 낮은 거부터 오름차순 정렬
display(resultDF.sort_values(by=['diff', 'neighbors']))

# 결과 최적의 K값: 5

,neighbors,train,valid,diff
4,5.0,0.975000,0.973333,0.001667
6,7.0,0.975000,0.973333,0.001667
11,12.0,0.975000,0.973333,0.001667
10,11.0,0.976667,0.973333,0.003333
12,13.0,0.970000,0.973333,0.003333
2,3.0,0.961667,0.966667,0.005000
5,6.0,0.968333,0.973333,0.005000
3,4.0,0.965000,0.960000,0.005000
14,15.0,0.975000,0.966667,0.008333
16,17.0,0.975000,0.966667,0.008333


### 최종 최적의 Hyper-Parameter : 5 => 최근접이웃 알고리즘 특성에 따라서 결정(스케일링은 안한 값, 해야 됨)


[4] 최종 모델 생성 <hr>
최종 모델이 일반화가 되어 있는지 확인

In [218]:
# --------------------------------------------------------------
# 전체 학습용 데이터셋과 결정된 하이퍼파라미터로 설정된 모델 생성
# --------------------------------------------------------------
# 1. 최종모델 생성
final_model = KNeighborsClassifier(n_neighbors=5)

# 2. 최종모델 훈련
final_model.fit(featureDF, targetSR)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[5] 최종 테스트용 데이터셋으로 평가 진행 <hr>
- 최종 테스트용 데이터셋의 성능이 좋아야 됨 => 일반화가 잘 되었다는 뜻
- 만약에 성능이 좋지 않다   -> 하이퍼파라미터 튜닝 또는 피쳐부터 다시 검토 진행
- 그럼에도 성능이 좋지 않다 -> 학습 알고리즘 변경